### Importing Libraries

In [32]:
import pandas as pd
import os
from glob import glob

# Function to load and combine CSVs from a folder
def load_all_csvs(folder_path):
    all_files = glob(os.path.join(folder_path, "*.csv"))
    df_list = [pd.read_csv(file) for file in all_files]
    return pd.concat(df_list, ignore_index=True)


### Function to clean the dataset

In [ ]:
# ✅ Reusable cleaning function for any index
def clean_index_data(df, save_as="cleaned_index.csv"):
    # 📛 1. Rename columns for consistency
    df.columns = [col.strip().lower() for col in df.columns]

    # 🧼 2. Remove duplicates (minute data might repeat)
    df.drop_duplicates(inplace=True)

    # ❓ 3. Check for missing values in price columns
    df.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)

    # 🗃️ 4. Convert date and time into a single datetime column
    df['date'] = df['date'].astype(str)  # Ensure it's string before combining
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y%m%d %H:%M', errors='coerce')
    df = df[df['datetime'].notnull()]  # Drop bad datetime conversions

    # 🧮 5. Ensure price columns are numeric
    for col in ['open', 'high', 'low', 'close']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)

    # 🔢 6. Add derived fields
    df['minute_return'] = df['close'].pct_change()  # return per minute
    df['intraday_range'] = df['high'] - df['low']   # candle size

    # 📊 7. Add datetime breakdown for aggregation in Power BI
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour

    # 📉 8. Remove junk data (if OHLC values are same)
    df = df[~((df['open'] == df['high']) & (df['high'] == df['low']) & (df['low'] == df['close']))]
    df=df.drop(columns=['date'])
    # 💾 9. Save the cleaned dataset
    os.makedirs("cleaned", exist_ok=True)
    df.to_csv(f"cleaned/{save_as}", index=False)

    print(f"✅ Cleaned data saved to cleaned/{save_as}")
    return df


### Reading the raw data

In [38]:
bank_df_raw = load_all_csvs("data/banknifty")
bank_df_raw.head()

,Instrument,Date,Time,Open,High,Low,Close
0,BANKNIFTY,20100302,09:01,8826.90,8856.95,8826.90,8839.80
1,BANKNIFTY,20100302,09:02,8840.75,8853.90,8832.80,8852.30
2,BANKNIFTY,20100302,09:03,8855.70,8855.70,8844.65,8846.65
3,BANKNIFTY,20100302,09:04,8843.85,8850.70,8843.75,8846.70
4,BANKNIFTY,20100302,09:05,8846.40,8846.40,8837.75,8841.60


### Calling clean data function

In [35]:
bank_df_clean = clean_index_data(bank_df_raw)
bank_df_clean.to_csv("cleaned/bank_nifty_cleaned.csv", index=False)


✅ Cleaned data saved to cleaned/cleaned_index.csv


### Cleaned data

In [37]:
df=pd.read_csv("cleaned/bank_nifty_cleaned.csv")
df.head()

,instrument,time,open,high,low,close,datetime,minute_return,intraday_range,year,month,day,hour
0,BANKNIFTY,09:01,8826.90,8856.95,8826.90,8839.80,2010-03-02 09:01:00,NaN,30.05,2010,3,2,9
1,BANKNIFTY,09:02,8840.75,8853.90,8832.80,8852.30,2010-03-02 09:02:00,0.001414,21.10,2010,3,2,9
2,BANKNIFTY,09:03,8855.70,8855.70,8844.65,8846.65,2010-03-02 09:03:00,-0.000638,11.05,2010,3,2,9
3,BANKNIFTY,09:04,8843.85,8850.70,8843.75,8846.70,2010-03-02 09:04:00,0.000006,6.95,2010,3,2,9
4,BANKNIFTY,09:05,8846.40,8846.40,8837.75,8841.60,2010-03-02 09:05:00,-0.000576,8.65,2010,3,2,9
